# EP07: 정보 보안의 끝판왕, 인텔 SGX 클러스터 만들기

## 애저 로그인 상태 확인

우선 애저 커맨드 라인 도구의 로그인 상태를 먼저 확인합니다. 로그인 상태가 아니면 다른 터미널에서 `az login` 명령을 실행합니다. 설치되어있지 않다면 OS 버전 별로 애저 커맨드 라인 도구를 설치하고 준비합니다.

In [1]:
az account list

The below script needs to be able to find the current output cell; this is an easy method to get it.

Name               CloudName    SubscriptionId                        State    IsDefault
-----------------  -----------  ------------------------------------  -------  -----------
rkttu@outlook.com  AzureCloud   e6535fda-2c47-411d-9e51-ae11ef7bde27  Enabled  True


## AKS 엔진 설치 상태 확인

아래 명령어를 실행하여 `aks-engine`이 설치되었는지 확인합니다. 설치되어있지 않다면 OS 버전 별로 각각 설치 가이드를 따라 설치를 진행합니다.

In [2]:
aks-engine version

Version: v0.56.0
GitCommit: 666073d49
GitTreeState: clean


## 서비스 주체 생성

쿠버네티스 클러스터를 만든 후에는 클러스터 수준에서 스스로 애저 리소스들을 제어할 수 있어야 하므로, 서비스 주체 계정을 만들어 계속 해당 계정을 사용하도록 설정합니다.

In [3]:
$SubscriptionData = (az account list -o json) | ConvertFrom-Json -Depth 16
$SubscriptionId = $SubscriptionData[0].id
$SvcPrincipal = (az ad sp create-for-rbac `
  -n="AzurePlaylist" --role="Contributor" `
  --scopes="/subscriptions/$SubscriptionId" `
  -o json) | ConvertFrom-Json -Depth 16

$PrincipalId = $SvcPrincipal.AppId
$PrincipalPwd = $SvcPrincipal.Password

"Your App ID is $PrincipalId"

Changing "AzurePlaylist" to a valid URI of "http://AzurePlaylist", which is the required format used for service principal names
Found an existing application instance of "5f6ed45a-660e-47f5-9c08-6e5515f17640". We will patch it
Creating a role assignment under the scope of "/subscriptions/e6535fda-2c47-411d-9e51-ae11ef7bde27"
  Role assignment already exists.

Your App ID is 5f6ed45a-660e-47f5-9c08-6e5515f17640


## SSH 키 준비하기

아래 명령어를 실행하여 SSH 키의 공개 키를 준비합니다. 설치되어있지 않다면 SSH 키 쌍을 만듭니다.

In [4]:
$PublicKeyValue = Get-Content -Path "$HOME\.ssh\id_rsa.pub"

## API 모델 준비하기

이제 API 모델을 준비할 차례입니다. AKS 엔진은 코드 기반 인프라 관리 방식을 구현하기 위하여 JSON API 모델을 작성하면, 여기에 맞추어 ARM 템플릿을 만들어줍니다. 이것을 애저 명령줄 도구를 통해 배포하면 손쉽게 쿠버네티스 클러스터를 생성할 수 있습니다.

In [5]:
$ApiModel = @"
{
  'apiVersion': 'vlabs',
  'properties': {
    'orchestratorProfile': {
      'orchestratorType': 'Kubernetes'
    },
    'servicePrincipalProfile': {
      'clientId': '',
      'secret': ''
    },
    'masterProfile': {
      'count': 1,
      'dnsPrefix': '',
      'vmSize': 'Standard_D2_v3'
    },
    'agentPoolProfiles': [
      {
        'name': 'linuxpool1',
        'count': 3,
        'distro': 'ubuntu-18.04-gen2',
        'vmSize': 'Standard_DC2s_v2'
      },
      {
        'name': 'linuxpool2',
        'count': 3,
        'distro': 'ubuntu-18.04-gen2',
        'vmSize': 'Standard_DC2s_v2'
      }
    ],
    'linuxProfile': {
      'adminUsername': 'azureuser',
      'ssh': {
        'publicKeys': [
          {
            'keyData': ''
          }
        ]
      }
    }
  }
}
"@ | ConvertFrom-Json -Depth 16

## API 모델 내용 변경하기

나중에 리눅스 마스터 노드에 접속할 수 있도록 현재 컴퓨터에서 사용하는 SSH 공개 키를 배포할 때 같이 등록합니다. 그리고 서비스 주체를 가리키는 아이디 값과 고유 비밀 키 값을 API 모델에 같이 지정합니다.

In [6]:
$ApiModel.properties.linuxProfile.ssh.publicKeys[0].keyData = "$PublicKeyValue"
$ApiModel.properties.servicePrincipalProfile.clientId = "$PrincipalId"
$ApiModel.properties.servicePrincipalProfile.secret = "$PrincipalPwd"

In [7]:
$ApiModel | ConvertTo-Json -Depth 16 | Out-File -Encoding utf8 -Path 'ep07.json' -Force

dir 'ep07*'



    Directory: /Users/rkttu/Projects/azure-playlist-aks-engine

Mode                 LastWriteTime         Length Name
----                 -------------         ------ ----
-----     2020. 11. 15.  오후 8:58          19671 EP07.ipynb
-----     2020. 11. 15. 오후 10:47           1291 ep07.json



## API 모델을 ARM 템플릿으로 변환하고 배포하기

다음의 명령어를 실행하여 API 모델을 ARM 템플릿으로 변환하고 곧바로 애저에 배포하도록 하겠습니다. 이 명령어는 최초에 한 번만 실행하고, 그 이후에는 `ep03_output` 디렉터리 안에 있는 파일들을 이용하여 클러스터를 관리하도록 합니다.

In [11]:
$DnsPrefix = 'rkttuep07'
$ResourceGroup = 'rkttuep07'
$Location = 'westus'

In [17]:
aks-engine deploy --dns-prefix "$DnsPrefix" `
    --resource-group "$ResourceGroup" `
    --location "$Location" `
    --api-model "ep07.json" `
    --auto-suffix `
    --output-directory "ep07_output" `
    --auth-method cli

INFO[0000] No subscription provided, using selected subscription from azure CLI: e6535fda-2c47-411d-9e51-ae11ef7bde27 
WARN[0003] apimodel: missing masterProfile.dnsPrefix will use "rkttuep07" 
WARN[0005] containerd will be upgraded to version 1.3.7 
INFO[0009] Starting ARM Deployment rkttuep07-956989609 in resource group rkttuep07. This will take some time... 
Error: resources.DeploymentsClient#CreateOrUpdate: Failure sending request: StatusCode=400 -- Original Error: Code="InvalidTemplateDeployment" Message="The template deployment 'rkttuep07-956989609' is not valid according to the validation procedure. The tracking id is '47b8edb5-7cb9-42a6-8155-d3080d040e93'. See inner errors for details." Details=[{"code":"QuotaExceeded","message":"Operation could not be completed as it results in exceeding approved standardDCSv2Family Cores quota. Additional details - Deployment Model: Resource Manager, Location: westus, Current Limit: 0, Current Usage: 0, Additional Required: 12, (Minimum) New 

## kubectl 실행해보기

클러스터가 잘 만들어졌는지 확인해보기 위하여 다음과 같이 `KUBECONFIG` 환경 변수를 설정하여 클러스터에 접근하고, 노드와 구성 상태를 확인합니다.

In [17]:
$env:KUBECONFIG="ep07_output/kubeconfig/kubeconfig.$Location.json"
kubectl get nodes -o wide

NAME                        STATUS   ROLES    AGE    VERSION   INTERNAL-IP    EXTERNAL-IP   OS-IMAGE             KERNEL-VERSION     CONTAINER-RUNTIME
k8s-linuxpool1-28379872-0   Ready    agent    118s   v1.18.8   10.240.0.34    <none>        Ubuntu 18.04.5 LTS   5.4.0-1025-azure   docker://19.3.12
k8s-master-28379872-0       Ready    master   118s   v1.18.8   10.255.255.5   <none>        Ubuntu 18.04.5 LTS   5.4.0-1025-azure   docker://19.3.12


In [21]:
$env:NAMESPACE='default'
$env:K8S_SECRET_NAME='dashboard-grafana'
kubectl get secret $env:K8S_SECRET_NAME -o jsonpath="{.data.grafana-admin-user}" | base64 --decode

Error from server (NotFound): secrets "dashboard-grafana" not found


## 리소스 그룹 정리

테스트를 위해 생성한 리소스 그룹을 제거하고 모든 리소스를 일괄 정리합니다.

In [22]:
az group delete --name $ResourceGroup --yes --no-wait

## 감사합니다!